# Classification 2.1 Reuters data set.

# Loading the needed libraries.

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from tqdm import tqdm_notebook

import os
import itertools

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.multiclass import OneVsRestClassifier

from sklearn.decomposition import IncrementalPCA as iPCA

from sklearn.metrics import f1_score, precision_score, recall_score, brier_score_loss

from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split

In [36]:
import nltk
nltk.download("stopwords")
nltk.download("reuters")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/dlab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to /home/dlab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /home/dlab-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
 
cachedStopWords = stopwords.words("english")

# Loading the data and taking the first look.

The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" and "test".
This split is for training and testing algorithms that automatically detect the topic of a document.

In [28]:
from nltk.corpus import reuters 

 
def collection_stats():
    
    # List of documents
    documents = reuters.fileids()
    print(str(len(documents)) + " documents")
 
    train_docs = list(filter(lambda doc: doc.startswith("train"),
                        documents))
    print(str(len(train_docs)) + " total train documents")
 
    test_docs = list(filter(lambda doc: doc.startswith("test"),
                       documents));
    print(str(len(test_docs)) + " total test documents")
 
    # List of categories
    categories = reuters.categories()
    print(str(len(categories)) + " categories\n")
 
    # Documents in a category
    category_docs = reuters.fileids("acq")
 
    # Words for a document
    document_id = category_docs[0]
    document_words = reuters.words(category_docs[0])
    print(document_words, "\n")  
 
    # Raw document
    print(reuters.raw(document_id))
    
    document_id = category_docs[1]
    document_words = reuters.words(category_docs[1])
    print(document_words, "\n")  
 
    # Raw document
    print(reuters.raw(document_id))

See readers API descriptions
https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.api

https://www.nltk.org/api/nltk.corpus.reader.html?highlight=categorizedplaintextcorpusreader#nltk.corpus.reader.CategorizedPlaintextCorpusReader

In [29]:
collection_stats()

10788 documents
7769 total train documents
3019 total test documents
90 categories

['SUMITOMO', 'BANK', 'AIMS', 'AT', 'QUICK', 'RECOVERY', ...] 

SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER
  Sumitomo Bank Ltd &lt;SUMI.T> is certain to
  lose its status as Japan's most profitable bank as a result of
  its merger with the Heiwa Sogo Bank, financial analysts said.
      Osaka-based Sumitomo, with desposits of around 23.9
  trillion yen, merged with Heiwa Sogo, a small, struggling bank
  with an estimated 1.29 billion dlrs in unrecoverable loans, in
  October.
      But despite the link-up, Sumitomo President Koh Komatsu
  told Reuters he is confident his bank can quickly regain its
  position.
      "We'll be back in position in first place within three
  years," Komatsu said in an interview.
      He said that while the merger will initially reduce
  Sumitomo's profitability and efficiency, it will vastly expand
  Sumitomo's branch network in the Tokyo metropolitan area where
  it

In [7]:
reuters.categories()[:20]

['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr']

#### print ids of documents in category 'barley'

In [8]:
reuters.fileids('barley')

['test/15618',
 'test/15649',
 'test/15676',
 'test/15728',
 'test/15871',
 'test/15875',
 'test/15952',
 'test/17767',
 'test/17769',
 'test/18024',
 'test/18263',
 'test/18908',
 'test/19275',
 'test/19668',
 'training/10175',
 'training/1067',
 'training/11208',
 'training/11316',
 'training/11885',
 'training/12428',
 'training/13099',
 'training/13744',
 'training/13795',
 'training/13852',
 'training/13856',
 'training/1652',
 'training/1970',
 'training/2044',
 'training/2171',
 'training/2172',
 'training/2191',
 'training/2217',
 'training/2232',
 'training/3132',
 'training/3324',
 'training/395',
 'training/4280',
 'training/4296',
 'training/5',
 'training/501',
 'training/5467',
 'training/5610',
 'training/5640',
 'training/6626',
 'training/7205',
 'training/7579',
 'training/8213',
 'training/8257',
 'training/8759',
 'training/9865',
 'training/9958']

#### print categories of 'training/9865', 'training/9880' documents

In [26]:
reuters.categories(['training/9865', 'training/9880'])

['barley', 'corn', 'grain', 'money-fx', 'wheat']

#### calculate number of documents in each category

In [27]:
for i in reuters.categories():
    print(i, len(reuters.fileids(i)))

acq 2369
alum 58
barley 51
bop 105
carcass 68
castor-oil 2
cocoa 73
coconut 6
coconut-oil 7
coffee 139
copper 65
copra-cake 3
corn 237
cotton 59
cotton-oil 3
cpi 97
cpu 4
crude 578
dfl 3
dlr 175
dmk 14
earn 3964
fuel 23
gas 54
gnp 136
gold 124
grain 582
groundnut 9
groundnut-oil 2
heat 19
hog 22
housing 20
income 16
instal-debt 6
interest 478
ipi 53
iron-steel 54
jet 5
jobs 67
l-cattle 8
lead 29
lei 15
lin-oil 2
livestock 99
lumber 16
meal-feed 49
money-fx 717
money-supply 174
naphtha 6
nat-gas 105
nickel 9
nkr 3
nzdlr 4
oat 14
oilseed 171
orange 27
palladium 3
palm-oil 40
palmkernel 3
pet-chem 32
platinum 12
potato 6
propane 6
rand 3
rape-oil 8
rapeseed 27
reserves 73
retail 25
rice 59
rubber 49
rye 2
ship 286
silver 29
sorghum 34
soy-meal 26
soy-oil 25
soybean 111
strategic-metal 27
sugar 162
sun-meal 2
sun-oil 7
sunseed 16
tea 13
tin 30
trade 485
veg-oil 124
wheat 283
wpi 29
yen 59
zinc 34


# Preprocessing

In [30]:
def tokenize(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words
                  if word not in cachedStopWords]
    tokens = (list(map(lambda token: PorterStemmer().stem(token),
                  words)));
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter(lambda token:
                  p.match(token) and len(token)>=min_length,
         tokens))
    return filtered_tokens

stemmers https://pythonspot.com/nltk-stemming/

In [31]:
# Return the representer, without transforming
def tf_idf(docs):
    tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=3,
                        max_df=0.90, max_features=3000,
                        use_idf=True, sublinear_tf=True,
                        norm='l2')
    tfidf.fit(docs)
    return tfidf

TfIdf Sklearn API

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [33]:
def feature_values(doc, representer):
    doc_representation = representer.transform([doc])
    features = representer.get_feature_names()
    return [(features[index], doc_representation[0, index])
                 for index in doc_representation.nonzero()[1]]

In [34]:
def main():
    train_docs = []
    test_docs = []
 
    for doc_id in reuters.fileids():
        if doc_id.startswith("train"):
            train_docs.append(reuters.raw(doc_id))
        else:
            test_docs.append(reuters.raw(doc_id))
 
    representer = tf_idf(train_docs)
 
    for doc in test_docs:
        print(feature_values(doc, representer))

In [37]:
%%time
main()

[('yesterday', 0.04033607332946197), ('year', 0.04868306823970166), ('yasuhiro', 0.06784216189089363), ('would', 0.06372602853755906), ('worri', 0.06369058131200922), ('world', 0.06538908930353625), ('work', 0.04242957850539387), ('whose', 0.06074498801433533), ('whole', 0.05831497105507654), ('week', 0.03350388061036944), ('washington', 0.05100373242934736), ('warn', 0.05588698545333975), ('want', 0.04437615671705097), ('virtual', 0.06526893899506282), ('view', 0.0881170846232083), ('u.s.-japan', 0.0756981315596951), ('u.s.', 0.10359584116941481), ('two', 0.048942514648738625), ('trade', 0.11348195468326348), ('tough', 0.06585418728418185), ('told', 0.032587432290743255), ('tokyo', 0.09114601823204509), ('time', 0.03724208529585703), ('threat', 0.06130874824193495), ('third', 0.04526371139204408), ('textil', 0.06275911603898573), ('tax', 0.03718698709533435), ('tariff', 0.1358039419071181), ('talk', 0.040716751283530396), ('taiwan', 0.13419112859999516), ('surplu', 0.09585406131190968

[('u.k.', 0.11785846548405876), ('treasuri', 0.12388133819551933), ('transact', 0.11880758327727726), ('today', 0.15139648782248474), ('target', 0.12176023313927663), ('take-up', 0.1922749323286938), ('system', 0.1019280701890119), ('stg', 0.3087720409975042), ('shortag', 0.14122687741409493), ('said', 0.03969166052087025), ('rise', 0.09270330595729243), ('repurchas', 0.1446979597824553), ('remov', 0.15930284163496267), ('partli', 0.14924426072577804), ('outflow', 0.18107736073529634), ('offset', 0.14058210237761096), ('offici', 0.09029722002538582), ('note', 0.07709893133366005), ('money', 0.18750308215638772), ('mln', 0.13998630659140432), ('matur', 0.15264276326368714), ('market', 0.1601245831208498), ('main', 0.12868533876541136), ('liquid', 0.13366526395991726), ('hand', 0.1504322550552602), ('forecast', 0.19376431862453725), ('factor', 0.1315950389926404), ('exchequ', 0.16588694579163493), ('england', 0.13317356059249932), ('drain', 0.2919842996843851), ('deficit', 0.119991333468

[('yield', 0.05119680381166472), ('yen', 0.09370121798350785), ('year', 0.019567768089141537), ('would', 0.05375420165139874), ('work', 0.04069640844366121), ('within', 0.044346929782266814), ('weight', 0.06233374301717808), ('week', 0.05440980758063314), ('weak', 0.05307312116926693), ('watch', 0.06287948401882695), ('want', 0.07206622349876324), ('wall', 0.05880439890078563), ('volatil', 0.11077939356976965), ('vehicl', 0.06439336870027798), ('valu', 0.0658733520861444), ('unit', 0.07081417898561222), ('undervalu', 0.06887910661228186), ('u.s.', 0.07636356451196646), ('type', 0.06345734544463097), ('two', 0.027725471058470275), ('turn', 0.08968648787676195), ('trillion', 0.07260600243727912), ('trend', 0.05145639983161858), ('tradit', 0.05650239388667615), ('tough', 0.1325570931616725), ('today', 0.031797873566693856), ('three', 0.0536986634797611), ('think', 0.07960761425126044), ('term', 0.035667967227826614), ('target', 0.04329945805334975), ('take', 0.07412872829602751), ('succes

[('yesterday', 0.07749088851606001), ('year', 0.09352656077276085), ('world', 0.07419379740009992), ('virtual', 0.1253901943780356), ('use', 0.07570766516385052), ('unlik', 0.1757145796537999), ('unless', 0.10490420009754256), ('total', 0.06294447454409846), ('tonn', 0.1681053527242778), ('tight', 0.12189510384295231), ('term', 0.07144108073474571), ('taken', 0.0952061946337312), ('take', 0.07074960100160248), ('sterl', 0.10826050175475721), ('step', 0.10013287375092121), ('soviet', 0.16779295383728665), ('soon', 0.09853512282032029), ('slightli', 0.09758203668476337), ('significantli', 0.11175559228984051), ('shipment', 0.09165912703154751), ('shearson', 0.28018547203078126), ('see', 0.07226754599053446), ('said', 0.0478674868315883), ('rise', 0.11179865540828482), ('return', 0.09079328968177346), ('report', 0.05866967700884442), ('reduct', 0.09395141394986867), ('reduc', 0.07557696320246639), ('recov', 0.11287993103139106), ('rate', 0.06373699817376904), ('quarterli', 0.0862726029602

[('york', 0.15429283392445084), ('would', 0.09103081554547013), ('u.s.', 0.05441018629686205), ('trust', 0.2626523937529321), ('term', 0.07486724916235085), ('suspens', 0.1358608609363328), ('suspend', 0.1129785500732676), ('stop', 0.10710693666119472), ('statu', 0.1308388515633775), ('seven', 0.08579452459372906), ('second', 0.08749156552775837), ('said', 0.06217588658685179), ('resum', 0.11829342214648139), ('result', 0.11783059108354146), ('rest', 0.11321414044718095), ('requir', 0.0863662882118836), ('regul', 0.11393751057471223), ('reduc', 0.13409976274828433), ('receiv', 0.08331787775779448), ('reach', 0.0869543642259366), ('rate', 0.06679369452504759), ('quarter', 0.07269904249001233), ('put', 0.091787535407557), ('potenti', 0.10675753959590979), ('place', 0.09711226824792545), ('payment', 0.20428447360554494), ('new', 0.10140216325950006), ('net', 0.08380333110526106), ('negoti', 0.15002957453411483), ('mln', 0.07854712180872303), ('matur', 0.11393751057471223), ('long-term', 0

...

[('warrant', 0.1270807928305093), ('valu', 0.08861257562421541), ('use', 0.08608956308348857), ('unit', 0.06758907723180854), ('time', 0.13775151246033424), ('target', 0.09861949368745791), ('subsidiari', 0.08726306956859166), ('strateg', 0.13393377901047748), ('stock', 0.06537630095053691), ('step', 0.11386423465648048), ('spokesman', 0.08497456607114064), ('spin', 0.1621676343968994), ('specif', 0.11842944488460605), ('sharehold', 0.13823134968552356), ('servic', 0.14783801827841953), ('seriou', 0.1241717610822362), ('sell', 0.07892123941191644), ('second', 0.09493644788698342), ('sale', 0.13159715524700383), ('said', 0.08388871340534265), ('return', 0.10324390037257021), ('restructur', 0.10786369884135642), ('repurchas', 0.1171978663594621), ('reorgan', 0.14079472959357617), ('realiz', 0.1366211700961881), ('rather', 0.11660660657926844), ('railroad', 0.15573275433331407), ('program', 0.2027731826369881), ('proce', 0.12363274642304745), ('price', 0.06405734241510251), ('plan', 0.124

[('tender', 0.2604472248177535), ('share', 0.19856983908954345), ('said', 0.09078470605092427), ('restaur', 0.40626292792511876), ('respons', 0.1749755652699723), ('receiv', 0.1507878691526596), ('presid', 0.14079160571312216), ('offer', 0.12497901475110289), ('octob', 0.18432386372546647), ('novemb', 0.18850242436404618), ('martin', 0.24105915106091666), ('manag', 0.2394315134281837), ('major', 0.2222608384195711), ('led', 0.18583651249812158), ('industri', 0.11262098282353672), ('inc', 0.0848762491203987), ('group', 0.19870023160742523), ('give', 0.15642161875160843), ('extend', 0.1817005736992045), ('dlr', 0.07116815677906337), ('compani', 0.08738148519553508), ('class', 0.43462676405646167), ('chairman', 0.1445942965097454), ('associ', 0.14969781600686205)]
[('year', 0.13328375837118295), ('technolog', 0.3341446417907378), ('shr', 0.1778179212502252), ('share', 0.16965836921733823), ('sept', 0.4849755135915134), ('sale', 0.3175152810722934), ('qtr', 0.18954561269634057), ('prior', 

[('year', 0.05021177289858925), ('would', 0.04663534185261026), ('war', 0.05999842583583069), ('want', 0.04576961937192856), ('vulner', 0.07463849252829641), ('volum', 0.13944438026925787), ('visit', 0.06124228251274708), ('unsuccess', 0.07204985822805655), ('unlik', 0.09433620241831144), ('unilater', 0.07204985822805655), ('two', 0.02981392687214659), ('trader', 0.0996028099401362), ('trade', 0.031025226430140034), ('tonn', 0.09025102307073223), ('term', 0.10707710112359152), ('talk', 0.08813186182694825), ('supplier', 0.1592791308312254), ('suppli', 0.13729946134460352), ('summer', 0.061923846457740614), ('suggest', 0.0540885608841238), ('substitut', 0.07352582481554808), ('subject', 0.04631732752086004), ('strait', 0.07204985822805655), ('state', 0.036363365352460944), ('stabl', 0.059709076891862466), ('spot', 0.10287309555821242), ('southeast', 0.0696018997104449), ('sourc', 0.14269564621427122), ('sometim', 0.07301057258816412), ('sluggish', 0.0699724842535059), ('slightli', 0.052

[('year', 0.08667835535484808), ('worst', 0.10440202354430862), ('unlik', 0.0822119219417205), ('unemploy', 0.09111357231731235), ('two', 0.04399165547659668), ('tin', 0.1027004193613651), ('three', 0.050322259933200574), ('tax', 0.09582182599584677), ('take', 0.056046140348941446), ('surplu', 0.06951156152379158), ('studi', 0.07698944965371128), ('strongli', 0.09360101814754823), ('strateg', 0.0933041985651461), ('stood', 0.08669474421128923), ('state', 0.05365561696770534), ('spend', 0.1335057232700621), ('slash', 0.10928961030934632), ('sharpli', 0.07618501004119405), ('seven', 0.06485409847957616), ('semiconductor', 0.08688831883628789), ('sector', 0.12333845003930506), ('said', 0.07160457519126309), ('rubber', 0.09301292733848089), ('risen', 0.0942121771428011), ('ringgit', 0.28970707763035736), ('repay', 0.08650354479503268), ('recoveri', 0.08748390306070643), ('recov', 0.08942077930617418), ('recess', 0.09848860491258593), ('receiv', 0.06298194290138007), ('public', 0.0668740818

[('zealand', 0.19661932348723798), ('year', 0.0677359185338042), ('would', 0.10082070728480635), ('widespread', 0.1067643758064951), ('valu', 0.0641742160947401), ('urg', 0.08349061405990468), ('unsuccess', 0.11051937048652939), ('unlik', 0.08546512842441249), ('turn', 0.08737311631348144), ('thought', 0.09761945930269023), ('tax', 0.16424867585781772), ('tariff', 0.08233761714019604), ('take', 0.05826394115941628), ('speech', 0.09610441441526174), ('solv', 0.10196511429997898), ('sinc', 0.0601614407515815), ('short-term', 0.08896776418174233), ('servic', 0.06323487041816948), ('said', 0.0649979674423363), ('restor', 0.09894257593922237), ('reserv', 0.06133533500724791), ('relianc', 0.1163981397250464), ('reduc', 0.10538044118776003), ('ratio', 0.0982675126791015), ('rate', 0.08887143254938358), ('product', 0.04957466428483597), ('problem', 0.07246260698163014), ('probabl', 0.0778327196505323), ('primarili', 0.10001104053629296), ('price', 0.04639104219286648), ('prefer', 0.07556462097

In [43]:
# List of document ids
documents = reuters.fileids()
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
 
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

In [44]:
%%time
# Tokenisation
vectorizer = TfidfVectorizer(stop_words=cachedStopWords,
                             tokenizer=tokenize)
 
# Learn and transform train documents
vectorised_train_documents = vectorizer.fit_transform(train_docs)
vectorised_test_documents = vectorizer.transform(test_docs)
 
# Transform multilabel labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id)
                                  for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id)
                             for doc_id in test_docs_id])

CPU times: user 49.1 s, sys: 12.3 ms, total: 49.1 s
Wall time: 49.1 s


MultiLabelBinarizer API
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

# Incremental PCA

In [45]:
%%time
n = vectorised_train_documents.shape[0] #how many rows we have in the dataset

n_components = 2400
chunk_size = n_components + 100

ipca = iPCA(n_components=n_components) 

vectorised_train_documents_arr = vectorised_train_documents.toarray()
vectorised_test_documents_arr = vectorised_test_documents.toarray()
print(vectorised_train_documents_arr.shape)

for i in tqdm_notebook(range(0, n // chunk_size)):
    ipca.partial_fit(vectorised_train_documents_arr[i * chunk_size : (i + 1) * chunk_size])

ipca.partial_fit(vectorised_train_documents_arr[(i + 1) * chunk_size:])
    
print(np.sum(ipca.explained_variance_ratio_))

(7769, 20682)



0.8638125559454235
CPU times: user 23min 51s, sys: 1min 18s, total: 25min 10s
Wall time: 6min 36s


In [46]:
%%time
vectorised_train_ipca = ipca.transform(vectorised_train_documents_arr)
vectorised_test_ipca = ipca.transform(vectorised_test_documents_arr)

print(np.shape(vectorised_train_ipca), np.shape(vectorised_train_documents))

(7769, 2400) (7769, 20682)
CPU times: user 1min 27s, sys: 2.5 s, total: 1min 30s
Wall time: 23.7 s


# Modeling and scoring

## Helper functions

In [47]:
def classifier_f(clf, X_train, y_train, X_test):
    # Classifier
    classifier = OneVsRestClassifier(clf)
    classifier.fit(X_train, y_train)

    predictions = classifier.predict(X_test)
    return predictions



def eval_f(test_labels, predictions):
    precision = precision_score(test_labels, predictions,
                                average='micro')
    recall = recall_score(test_labels, predictions,
                          average='micro')
    f1 = f1_score(test_labels, predictions, average='micro')

    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}"
            .format(precision, recall, f1))

    precision = precision_score(test_labels, predictions,
                                average='macro')
    recall = recall_score(test_labels, predictions,
                          average='macro')
    f1 = f1_score(test_labels, predictions, average='macro')

    print("Macro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}"
            .format(precision, recall, f1))

## Logistic regression

In [49]:
C_values = [50, 100, 200, 500]

for c in tqdm_notebook(C_values):
    
    predictions_LR = classifier_f( LogisticRegression( C=c, random_state=42), vectorised_train_ipca, train_labels, vectorised_test_ipca)
    print("C = {0:2.2f}".format(c))
    eval_f(test_labels, predictions_LR)
    print("\n")

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



C = 50.00
Micro-average quality numbers
Precision: 0.9416, Recall: 0.7925, F1-measure: 0.8606
Macro-average quality numbers
Precision: 0.6184, Recall: 0.3612, F1-measure: 0.4328


C = 100.00
Micro-average quality numbers
Precision: 0.9399, Recall: 0.7975, F1-measure: 0.8629
Macro-average quality numbers
Precision: 0.6297, Recall: 0.3726, F1-measure: 0.4455


C = 200.00
Micro-average quality numbers
Precision: 0.9391, Recall: 0.7994, F1-measure: 0.8637
Macro-average quality numbers
Precision: 0.6222, Recall: 0.3800, F1-measure: 0.4491


C = 500.00
Micro-average quality numbers
Precision: 0.9361, Recall: 0.8024, F1-measure: 0.8641
Macro-average quality numbers
Precision: 0.6190, Recall: 0.3866, F1-measure: 0.4542





## SVM classifiers

SV Classifier sclearn API
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [50]:
%%time

predictions_LinearSVC = classifier_f( LinearSVC(random_state=42), vectorised_train_documents, train_labels, vectorised_test_documents)
eval_f(test_labels, predictions_LinearSVC)

Micro-average quality numbers
Precision: 0.9455, Recall: 0.8013, F1-measure: 0.8674
Macro-average quality numbers
Precision: 0.6493, Recall: 0.3948, F1-measure: 0.4665
CPU times: user 1.98 s, sys: 7.77 ms, total: 1.99 s
Wall time: 1.99 s


In [51]:
%%time

predictions_LinearSVC_ipca = classifier_f( LinearSVC(random_state=42), vectorised_train_ipca, train_labels, vectorised_test_ipca)
eval_f(test_labels, predictions_LinearSVC_ipca)

Micro-average quality numbers
Precision: 0.9479, Recall: 0.8021, F1-measure: 0.8689
Macro-average quality numbers
Precision: 0.6189, Recall: 0.3745, F1-measure: 0.4439
CPU times: user 52.2 s, sys: 14.4 s, total: 1min 6s
Wall time: 1min 4s


In [52]:
C_values = [ 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50, 100, 200]

for c in tqdm_notebook(C_values):
    
    predictions_LinearSVC = classifier_f( LinearSVC( C=c, random_state=42), vectorised_train_documents, train_labels, vectorised_test_documents)
    print("C = {0:2.2f}".format(c))
    eval_f(test_labels, predictions_LinearSVC)
    print("\n")

C = 0.05
Micro-average quality numbers
Precision: 0.9766, Recall: 0.6140, F1-measure: 0.7540
Macro-average quality numbers
Precision: 0.2671, Recall: 0.1041, F1-measure: 0.1354


C = 0.10
Micro-average quality numbers
Precision: 0.9680, Recall: 0.6782, F1-measure: 0.7975
Macro-average quality numbers
Precision: 0.3845, Recall: 0.1623, F1-measure: 0.2102


C = 0.50
Micro-average quality numbers
Precision: 0.9542, Recall: 0.7839, F1-measure: 0.8607
Macro-average quality numbers
Precision: 0.5660, Recall: 0.3419, F1-measure: 0.4095


C = 1.00
Micro-average quality numbers
Precision: 0.9455, Recall: 0.8013, F1-measure: 0.8674
Macro-average quality numbers
Precision: 0.6493, Recall: 0.3948, F1-measure: 0.4665


C = 2.00
Micro-average quality numbers
Precision: 0.9397, Recall: 0.8120, F1-measure: 0.8712
Macro-average quality numbers
Precision: 0.6791, Recall: 0.4379, F1-measure: 0.5078


C = 5.00
Micro-average quality numbers
Precision: 0.9351, Recall: 0.8154, F1-measure: 0.8712
Macro-averag

In [53]:
%%time

kernels = [ "rbf", "sigmoid"]  
C_values = [5e03, 1e04, 2e04, 5e04, 1e05, 2e05, 5e05]

for kern in kernels:
    for c in tqdm_notebook(C_values):
    
        predictions_SVC = classifier_f( SVC( C=c, kernel=kern, random_state=42), vectorised_train_documents, train_labels, vectorised_test_documents)
        print("Kernel: {}, C = {:.2f}".format(kern, c))
        eval_f(test_labels, predictions_SVC)
        print("\n")

Kernel: rbf, C = 5000.00
Micro-average quality numbers
Precision: 0.9577, Recall: 0.7559, F1-measure: 0.8449
Macro-average quality numbers
Precision: 0.4732, Recall: 0.2641, F1-measure: 0.3236


Kernel: rbf, C = 10000.00
Micro-average quality numbers
Precision: 0.9472, Recall: 0.7957, F1-measure: 0.8649
Macro-average quality numbers
Precision: 0.5765, Recall: 0.3701, F1-measure: 0.4335


Kernel: rbf, C = 20000.00
Micro-average quality numbers
Precision: 0.9369, Recall: 0.8176, F1-measure: 0.8732
Macro-average quality numbers
Precision: 0.6714, Recall: 0.4362, F1-measure: 0.5057


Kernel: rbf, C = 50000.00
Micro-average quality numbers
Precision: 0.9298, Recall: 0.8170, F1-measure: 0.8698
Macro-average quality numbers
Precision: 0.6871, Recall: 0.4629, F1-measure: 0.5316


Kernel: rbf, C = 100000.00
Micro-average quality numbers
Precision: 0.9261, Recall: 0.8170, F1-measure: 0.8682
Macro-average quality numbers
Precision: 0.6905, Recall: 0.4625, F1-measure: 0.5299


Kernel: rbf, C = 200

In [54]:
%%time

predictions_SVC_poly = classifier_f( SVC( C=1e04, gamma=0.01, kernel="poly", random_state=42), vectorised_train_documents, train_labels, vectorised_test_documents)
eval_f(test_labels, predictions_SVC_poly)

Micro-average quality numbers
Precision: 1.0000, Recall: 0.0331, F1-measure: 0.0641
Macro-average quality numbers
Precision: 0.0111, Recall: 0.0013, F1-measure: 0.0023
CPU times: user 4min 51s, sys: 1.21 s, total: 4min 52s
Wall time: 4min 52s


understanding parameters

https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

## Naive Bayes classifiers.

In [55]:
%%time

predictions_GaussianNB_ipca = classifier_f( GaussianNB(), vectorised_train_ipca, train_labels, vectorised_test_ipca)

CPU times: user 33.1 s, sys: 14.5 s, total: 47.6 s
Wall time: 47.6 s


In [56]:
eval_f(test_labels, predictions_GaussianNB_ipca)

Micro-average quality numbers
Precision: 0.0806, Recall: 0.6854, F1-measure: 0.1442
Macro-average quality numbers
Precision: 0.1823, Recall: 0.4474, F1-measure: 0.1728


In [57]:
%%time

predictions_GaussianNB = classifier_f(  GaussianNB(), vectorised_train_documents_arr, train_labels, vectorised_test_documents_arr)

CPU times: user 4min 42s, sys: 2min 7s, total: 6min 50s
Wall time: 6min 50s


In [58]:
eval_f(test_labels, predictions_GaussianNB)

Micro-average quality numbers
Precision: 0.3563, Recall: 0.5200, F1-measure: 0.4228
Macro-average quality numbers
Precision: 0.0998, Recall: 0.1697, F1-measure: 0.1230


In [59]:
%%time

predictions_MultinomialNB = classifier_f( MultinomialNB(), vectorised_train_documents_arr, train_labels, vectorised_test_documents_arr)

CPU times: user 1min 23s, sys: 3.44 s, total: 1min 26s
Wall time: 1min 26s


In [60]:
eval_f(test_labels, predictions_MultinomialNB)

Micro-average quality numbers
Precision: 0.9938, Recall: 0.3397, F1-measure: 0.5064
Macro-average quality numbers
Precision: 0.0554, Recall: 0.0154, F1-measure: 0.0188


In [61]:
%%time

predictions_BernoulliNB = classifier_f( BernoulliNB(), vectorised_train_documents_arr, train_labels, vectorised_test_documents_arr)

CPU times: user 3min 37s, sys: 1min 27s, total: 5min 4s
Wall time: 5min 4s


In [62]:
eval_f(test_labels, predictions_BernoulliNB)

Micro-average quality numbers
Precision: 0.5976, Recall: 0.5296, F1-measure: 0.5616
Macro-average quality numbers
Precision: 0.0631, Recall: 0.0525, F1-measure: 0.0520


In [63]:
%%time

predictions_BernoulliNB_ipca = classifier_f( BernoulliNB(), vectorised_train_ipca, train_labels, vectorised_test_ipca)

CPU times: user 55 s, sys: 9.1 s, total: 1min 4s
Wall time: 1min 4s


In [64]:
eval_f(test_labels, predictions_BernoulliNB_ipca)

Micro-average quality numbers
Precision: 0.8668, Recall: 0.7316, F1-measure: 0.7935
Macro-average quality numbers
Precision: 0.6909, Recall: 0.5161, F1-measure: 0.5718
